MK užduotis 1-5
1. Sugeneruokite pseudoatsitiktinių skaičių sekas tiesiniu kongruentiniu metodu su maksimaliu periodu, kai modulis $m=776$ ir $m=1107$. Daugiklius $a$ parinkite taip, kad galingumai būtų didžiausi. Prieauglio $c$ parinkimui naudokitės gretimų narių koreliacija (teoriniai testai).

Tiesinio kongruentinio metodo formulė:
$X_{n+1} \equiv (aX_n + c) \pmod m, n \geq 0$.

Čia skaičiai:
$X_0$ - pradinė reikšmė, $X_0 \geq 0$,
$a$ - daugiklis, $a \geq 0$,
$c$ - prieauglis, $c \geq 0$,
$m$ - modulis, $m > X_0, m > a, m > c$, yra perenkami.

Gretimų narių koeficientas apibrežiamas taip:
$$
C := \frac{n \sum_{i=0}^{n-1} (U_i V_i) - \left(\sum_{i=0}^{n-1} U_i\right) \left(\sum_{i=0}^{n-1} V_i\right)}{\sqrt{\left(n \sum_{i=0}^{n-1} U_i^2 - \left(\sum_{i=0}^{n-1} U_i\right)^2\right) \left(n \sum_{i=0}^{n-1} V_i^2 - \left(\sum_{i=0}^{n-1} V_i\right)^2\right)}}
$$

Tiesinės kongruentinės sekos $(X_0, a, c, m)$ su maksimaliu periodu gretimų narių koreliacijos koeficientas

$$C \approx \frac{1}{a} \left(1 - 6 \frac{c}{m} + 6 \left(\frac{c}{m}\right)^2\right).$$
Šios apytikslės formulės klaida yra nedidesnė už $(a+6)/m$.

Kad tiesinis kongruentinis generatorius turėtų maksimalų periodą lygų $m$, turi būti tenkinamos šios sąlygos:

Skaičiai $c$ ir $m$ turi būti tarpusavyje pirminiai (jų DBD = 1).
Jei $p$ yra bet kuris pirminis skaičius, kuris dalijasi iš $m$, tada $(a-1)$ turi dalintis iš $p$.
Jei $m$ yra dalus iš 4, tada $(a-1)$ taip pat turi būti dalus iš 4.

Modulių faktorizacija

Modulis $776 = 8 \times 97 = 2^3 \times 97$

Modulis $1107 = 3 \times 369 = 3^3 \times 41$
